# Topic modelling

## 0.0 Imports

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dglover\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 1.0 Run

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['use', 'show', 'however', 'approach', 'well', 'provide',' present', 'include', 'word', 'nlp'])

In [4]:
# Import Dataset
df = pd.read_csv('..\\data\\arxiv_papers_full_v2.csv', index_col=0)
#print(df.target_names.unique())
df.head()

,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID
0,Natural Language Processing using Hadoop and K...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1
1,Integrating AI Planning with Natural Language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2
2,Simple Natural Language Processing Tools for D...,http://arxiv.org/pdf/1906.11608v2,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,This technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2
3,Towards the Study of Morphological Processing ...,http://arxiv.org/pdf/2006.16212v1,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,There is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1
4,Natural Language Understanding with Distribute...,http://arxiv.org/pdf/1511.07916v1,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,This is a lecture note for the course DS-GA 30...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1


In [49]:
df.isna().sum()

Title                  0
PDF URL                0
Author                 0
DOI                 3155
Published Date         0
Summary                0
Journal Ref         3043
Primary Category       0
Category               0
Entry ID               0
dtype: int64

In [5]:
df.shape

(3558, 10)

In [6]:
df['Summary'] = df['Summary'].map(lambda x: x.lower().replace('natural langauge processing', ''))

In [7]:
# Convert to list
data = df['Summary'].values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

C:\Users\dglover\Anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [50]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=40, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [51]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(1,
  '0.000*"sphere" + 0.000*"linguistics" + 0.000*"poll" + 0.000*"ascribing" + '
  '0.000*"consolidate" + 0.000*"watermark" + 0.000*"watermarke" + '
  '0.000*"watermarking" + 0.000*"opportunitie" + 0.000*"decentralized"'),
 (27,
  '0.000*"sphere" + 0.000*"linguistics" + 0.000*"poll" + 0.000*"ascribing" + '
  '0.000*"consolidate" + 0.000*"watermark" + 0.000*"watermarke" + '
  '0.000*"watermarking" + 0.000*"opportunitie" + 0.000*"decentralized"'),
 (21,
  '0.000*"sphere" + 0.000*"linguistics" + 0.000*"poll" + 0.000*"ascribing" + '
  '0.000*"consolidate" + 0.000*"watermark" + 0.000*"watermarke" + '
  '0.000*"watermarking" + 0.000*"opportunitie" + 0.000*"decentralized"'),
 (5,
  '0.000*"sphere" + 0.000*"linguistics" + 0.000*"poll" + 0.000*"ascribing" + '
  '0.000*"consolidate" + 0.000*"watermark" + 0.000*"watermarke" + '
  '0.000*"watermarking" + 0.000*"opportunitie" + 0.000*"decentralized"'),
 (8,
  '0.415*"augmentation" + 0.035*"data" + 0.031*"impose" + 0.000*"watermark" + '
  '0.000*

In [52]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -15.106614652163174

Coherence Score:  0.44816456715097963


In [53]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\dglover\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.292944 -0.306476       1        1  48.283723
16     0.317920 -0.177581       2        1  16.045024
13     0.265352 -0.114300       3        1   8.068654
23     0.277799 -0.124654       4        1   6.046323
36     0.189222  0.142870       5        1   2.525465
34     0.120460  0.124157       6        1   1.581105
9      0.141335  0.105003       7        1   1.520016
6      0.086271  0.090251       8        1   1.237999
10     0.109757  0.121480       9        1   1.197801
39     0.098251  0.109266      10        1   1.129023
17     0.076621  0.095945      11        1   1.055442
29     0.073402  0.078811      12        1   1.014495
18     0.077003  0.081920      13        1   0.983844
11     0.085307  0.068462      14        1   0.932839
38     0.054190  0.070356      15        1   0.850714
15     0.040415  0.063302      16        1   0.835941
37     0.013941  0.031814      17        1   0.631161
2      0.014518  0.027595      18        1   0.591835
33     0.000216  0.020268      19        1   0.563036
28    -0.022897  0.022673      20        1   0.521146
0     -0.026191  0.020521      21        1   0.519084
14    -0.002944  0.036566      22        1   0.513651
26    -0.043041  0.006126      23        1   0.437903
20    -0.052996  0.003730      24        1   0.400200
35    -0.073152 -0.008138      25        1   0.363862
3     -0.100814 -0.022957      26        1   0.293067
25    -0.079982 -0.011976      27        1   0.279339
7     -0.082088 -0.013069      28        1   0.268560
22    -0.111513 -0.028232      29        1   0.249105
31    -0.108125 -0.026594      30        1   0.236700
12    -0.131495 -0.037290      31        1   0.215431
19    -0.135604 -0.039040      32        1   0.172741
30    -0.155053 -0.046586      33        1   0.138837
32    -0.148560 -0.044202      34        1   0.122297
24    -0.168372 -0.050848      35        1   0.077028
8     -0.152261 -0.051007      36        1   0.070463
5     -0.184959 -0.054542      37        1   0.006583
27    -0.184959 -0.054542      38        1   0.006537
1     -0.184959 -0.054542      39        1   0.006513
21    -0.184959 -0.054542      40        1   0.006513, topic_info=            Term        Freq        Total Category  logprob  loglift
93         model  8672.00000  8672.000000  Default  30.0000  30.0000
114         text  2904.00000  2904.000000  Default  29.0000  29.0000
570       system  2098.00000  2098.000000  Default  28.0000  28.0000
159         task  4973.00000  4973.000000  Default  27.0000  27.0000
90         learn  1569.00000  1569.000000  Default  26.0000  26.0000
..           ...         ...          ...      ...      ...      ...
25         large     0.00181  1670.867834  Topic40  -9.2906  -4.0965
26          many     0.00181   962.196771  Topic40  -9.2906  -3.5446
27       natural     0.00181  4895.299211  Topic40  -9.2906  -5.1714
28   performance     0.00181  2101.330489  Topic40  -9.2906  -4.3257
29      platform     0.00181   165.713303  Topic40  -9.2906  -1.7856

[1815 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
47        1  0.536015      ability
47        2  0.067002      ability
47        7  0.393890      ability
189       7  0.978812  abstraction
1602     18  0.975548   accelerate
...     ...       ...          ...
277       3  0.999394         word
118       1  0.998377         work
118       2  0.001142         work
220      28  0.994188         year
907       7  0.992380    zero_shot

[852 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 17, 14, 24, 37, 35, 10, 7, 11, 40, 18, 30, 19, 12, 39, 16, 38, 3, 34, 29, 1, 15, 27, 21, 36, 4, 26, 8, 23, 32, 13, 20, 31, 33, 25, 9, 6, 28, 2, 22])

In [54]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

C:\Users\dglover\AppData\Local\Temp\ipykernel_16032\3153082663.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [55]:
df_dominant_topic.to_csv('../data/Dominant_topics_of_absracts.csv')

In [56]:
df_dominant_topic.tail(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3548,3548,4,0.4105,"language, natural, use, processing, base, method, dataset, datum, paper, work","[self_supervise, learning, speech, representation, successfully, apply, various, downstream, tas..."
3549,3549,4,0.5064,"language, natural, use, processing, base, method, dataset, datum, paper, work","[large, language, model, llm, revolutionize, natural, language, processing, demand, massive, res..."
3550,3550,4,0.4629,"language, natural, use, processing, base, method, dataset, datum, paper, work","[text, summarization, essential, task, natural, language, processing, researcher, develop, vario..."
3551,3551,4,0.4675,"language, natural, use, processing, base, method, dataset, datum, paper, work","[sentiment_analysis, vital, tool, uncover, insight, financial, article, news, social_media, shap..."
3552,3552,4,0.4984,"language, natural, use, processing, base, method, dataset, datum, paper, work","[recent, research, demonstrate, task, fine_tune, multi_modal, large, language, model, llm, use, ..."
3553,3553,4,0.5388,"language, natural, use, processing, base, method, dataset, datum, paper, work","[large, document, write, juridical, language, difficult, interpret, long, sentence, lead, intric..."
3554,3554,4,0.5120,"language, natural, use, processing, base, method, dataset, datum, paper, work","[promise, performance, various, natural, language, processing, task, current, system, vulnerable..."
3555,3555,4,0.4581,"language, natural, use, processing, base, method, dataset, datum, paper, work","[recent, advancement, speech, emotion, recognition, ser, model, state, art, deep, learning, appr..."
3556,3556,4,0.4898,"language, natural, use, processing, base, method, dataset, datum, paper, work","[recent_advance, large, language, model, lead, renew, interest, natural, language, processing, h..."
3557,3557,4,0.4608,"language, natural, use, processing, base, method, dataset, datum, paper, work","[pre_traine, model, czech, natural, language, processing, often, evaluate, purely, linguistic, t..."


In [57]:
df_dominant_topic['Dominant_Topic'].value_counts()

4     3552
16       6
Name: Dominant_Topic, dtype: int64

In [58]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,4,0.7407,"language, natural, use, processing, base, method, dataset, datum, paper, work","[application, domain, digital_humanitie, tool, chatbot, involve, form, process, natural, languag..."
1,16,0.4118,"model, task, performance, training, domain, state, train, achieve, representation, art","[recent, progress, natural, language, processing, domain, give, several, state, art, sota, pretr..."


In [59]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,4,0.7407,"language, natural, use, processing, base, method, dataset, datum, paper, work","[application, domain, digital_humanitie, tool, chatbot, involve, form, process, natural, languag..."
1,16,0.4118,"model, task, performance, training, domain, state, train, achieve, representation, art","[recent, progress, natural, language, processing, domain, give, several, state, art, sota, pretr..."


In [60]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
1,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
2,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
3,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
4,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",3552.0,0.9983
...,...,...,...,...
3553,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
3554,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
3555,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
3556,4,"language, natural, use, processing, base, method, dataset, datum, paper, work",NaN,NaN
